In [1]:
import pandas as pd
import numpy as np

# Importing dataset


In [2]:
!kaggle datasets download -d notshrirang/spotify-million-song-dataset

spotify-million-song-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
# %unzip spotify-million-song-dataset.zip

In [2]:
spotifyDF = pd.read_csv("spotifyDFMS3.csv")

# Milestone 3

In [3]:
from transformers import AutoTokenizer, TFT5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = TFT5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [4]:
spotifyDFMS3=spotifyDF.copy()

In [5]:
def split_sentences(df, col_name):
    df['X'] = df[col_name].apply(lambda x: ' '.join(x.split()[:len(x.split())//2]))
    df['Y'] = df[col_name].apply(lambda x: ' '.join(x.split()[len(x.split())//2:]))
    return df

spotifyDFMS3=split_sentences(spotifyDFMS3,"text")
spotifyDFMS3.head(2)

,artist,song,link,text,X,Y
0,Bob Dylan,4Th Time Around,/b/bob+dylan/4th+time+around_20021421.html,"When she said, ""Don't waste your words, they'r...","When she said, ""Don't waste your words, they'r...","I waited in the hallway, she went to get it, A..."
1,Bob Dylan,A Satisfied Mind,/b/bob+dylan/a+satisfied+mind_20021501.html,How many times have you heard someone say \r\...,How many times have you heard someone say If I...,"suddenly it happened Hmm, I lost every dime Bu..."


In [6]:
print(len(spotifyDFMS3["Y"][0])+len(spotifyDFMS3["X"][0]))
print(len(spotifyDFMS3["text"][0]))

1338
1482


There is a difference in the values since the split_sentences method also removes the extra unneeded tokens such as white spaces adn \n.

In [7]:
spotifyDFMS3['artist_lyric']=spotifyDFMS3["artist"]+" || "+spotifyDFMS3["X"]

In [8]:
spotifyDFMS3.head(2)

,artist,song,link,text,X,Y,artist_lyric
0,Bob Dylan,4Th Time Around,/b/bob+dylan/4th+time+around_20021421.html,"When she said, ""Don't waste your words, they'r...","When she said, ""Don't waste your words, they'r...","I waited in the hallway, she went to get it, A...","Bob Dylan || When she said, ""Don't waste your ..."
1,Bob Dylan,A Satisfied Mind,/b/bob+dylan/a+satisfied+mind_20021501.html,How many times have you heard someone say \r\...,How many times have you heard someone say If I...,"suddenly it happened Hmm, I lost every dime Bu...",Bob Dylan || How many times have you heard som...


In [9]:
def max_words(df, col_name):
    max_word_count = df[col_name].apply(lambda x: len(x.split(" "))).max()
    return max_word_count

max_word_count_X = max_words(spotifyDFMS3, 'X')
max_word_count_Y = max_words(spotifyDFMS3, 'Y')
max_word_count = max_words(spotifyDFMS3, 'artist_lyric')
print("X max :", max_word_count_X)
print("Y max :", max_word_count_Y)
print("X AND artist max:",max_word_count)

X max : 396
Y max : 397
X AND artist max: 399


In [11]:
from datasets import load_dataset
spotifyDataset = load_dataset('csv',data_files="spotifyDFMS3.csv", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
type(spotifyDataset)

datasets.arrow_dataset.Dataset

In [13]:
def split_sentences(df,OldDF):
    x=[]
    y=[]
    artist_lyric=[]
    for string in OldDF["X"]:
        x.append(string)
    for string in OldDF["Y"]:
        y.append(string)
    for string in OldDF["artist_lyric"]:
        artist_lyric.append(string)
    df = df.add_column("X", x)
    df = df.add_column("Y", y)
    df = df.add_column("artist_lyric", artist_lyric)
    return df

spotifyDataset=split_sentences(spotifyDataset,spotifyDFMS3)

In [14]:
spotifyDataset[0]

{'artist': 'Bob Dylan',
 'song': '4Th Time Around',
 'link': '/b/bob+dylan/4th+time+around_20021421.html',
 'text': 'When she said, "Don\'t waste your words, they\'re just lies,"  \r\nI cried she was deaf.  \r\nAnd she worked on my face until breaking my eyes,  \r\nThen said, "What else you got left?"  \r\nIt was then that I got up to leave  \r\nBut she said, "Don\'t forget,  \r\nEverybody must give something back  \r\nFor something they get."  \r\n  \r\nI stood there and hummed,  \r\nI tapped on her drum  \r\nAnd asked her how come.  \r\nAnd she buttoned her boot,  \r\nAnd straightened her suit,  \r\nThen she said, "Don\'t get cute."  \r\nSo I forced my hands in my pockets  \r\nAnd felt with my thumbs,  \r\nAnd gallantly handed her  \r\nMy very last piece of gum.  \r\n  \r\nShe threw me outside,  \r\nI stood in the dirt where ev\'ryone walked.  \r\nAnd after finding I\'d forgotten my shirt,  \r\nI went back and knocked.  \r\nI waited in the hallway, she went to get it,  \r\nAnd I trie

In [15]:
columns_to_drop = ["song", "link","text"]
spotifyDataset = spotifyDataset.remove_columns(columns_to_drop)

In [16]:
spotifyDataset[0]

{'artist': 'Bob Dylan',
 'X': 'When she said, "Don\'t waste your words, they\'re just lies," I cried she was deaf. And she worked on my face until breaking my eyes, Then said, "What else you got left?" It was then that I got up to leave But she said, "Don\'t forget, Everybody must give something back For something they get." I stood there and hummed, I tapped on her drum And asked her how come. And she buttoned her boot, And straightened her suit, Then she said, "Don\'t get cute." So I forced my hands in my pockets And felt with my thumbs, And gallantly handed her My very last piece of gum. She threw me outside, I stood in the dirt where ev\'ryone walked. And after finding I\'d forgotten my shirt, I went back and knocked.',
 'Y': 'I waited in the hallway, she went to get it, And I tried to make sense Out of that picture of you in your wheelchair That leaned up against-- Her Jamaican rum And when she did come, I asked her for some. She said, "No, dear." I said, "Your words aren\'t clear

In [17]:
spotifyDataset

Dataset({
    features: ['artist', 'X', 'Y', 'artist_lyric'],
    num_rows: 943
})

In [18]:
def process_dataset(example):
    artist = example['artist']
    first_half = example['X']
    second_half = example['Y']
    
    input_text = f"{artist}: {first_half}"
    target_text = second_half
    
    return {'input_text': input_text, 'target_text': target_text}

spotifyDataset = spotifyDataset.map(process_dataset, remove_columns=["X", "Y", "artist_lyric"])

Map:   0%|          | 0/943 [00:00<?, ? examples/s]

In [19]:
spotifyDataset

Dataset({
    features: ['artist', 'input_text', 'target_text'],
    num_rows: 943
})

In [20]:
def tokenize_function(examples):
    inputs = [f"<s> {text} </s>" for text in examples['input_text']]
    targets = [f"<s> {text} </s>" for text in examples['target_text']]
    model_inputs = tokenizer(inputs, max_length=max_word_count, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=max_word_count, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    decoder_input_ids = labels['input_ids'].copy()
    decoder_start_token_id = tokenizer.pad_token_id
    
    for idx in range(len(decoder_input_ids)):
        decoder_input_ids[idx] = [decoder_start_token_id] + decoder_input_ids[idx][:-1]
    
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["decoder_input_ids"] = decoder_input_ids

    return model_inputs

spotifyDataset = spotifyDataset.map(tokenize_function, batched=True, remove_columns=["artist", "input_text", "target_text"])

Map:   0%|          | 0/943 [00:00<?, ? examples/s]

In [21]:
spotifyDataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'],
    num_rows: 943
})

In [22]:
type(spotifyDataset)

datasets.arrow_dataset.Dataset

In [23]:
spotifyDataset=spotifyDataset.train_test_split(test_size=0.2)

In [24]:
spotifyDataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'],
        num_rows: 754
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'],
        num_rows: 189
    })
})

In [25]:
train_dataset = spotifyDataset["train"].with_format("numpy")[:] 
test_dataset = spotifyDataset["test"].with_format("numpy")[:]

In [39]:
import tensorflow as tf
from tensorflow import keras

optimizer = keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

In [29]:
# model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [32]:
# model.fit(train_dataset, validation_data=test_dataset, epochs=1)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert

 8/24 [=========>....................] - ETA: 58:20 - loss: 10.3900 - sparse_categorical_accuracy: 0.1490  

In [ ]:
# import tensorflow as tf
# from tensorflow import keras

# optimizer = keras.optimizers.Adam(learning_rate=5e-5)
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

In [27]:
def convert_to_tf_dataset(hf_dataset):
    return hf_dataset.to_tf_dataset(
        columns=['input_ids', 'attention_mask', 'decoder_input_ids', 'labels'],
        shuffle=True,
        batch_size=1
    )

train_dataset = convert_to_tf_dataset(spotifyDataset['train'])
eval_dataset = convert_to_tf_dataset(spotifyDataset['test'])

In [40]:
def train_step(batch):
    with tf.GradientTape() as tape:
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], decoder_input_ids=batch['decoder_input_ids'], labels=batch['labels'])
        loss = outputs.loss
        logits = outputs.logits
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    accuracy_metric.update_state(batch['labels'], logits)
    return loss

In [41]:
def eval_step(batch):
    outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], decoder_input_ids=batch['decoder_input_ids'], labels=batch['labels'])
    loss = outputs.loss
    logits = outputs.logits
    accuracy_metric.update_state(batch['labels'], logits)
    return loss

In [42]:
num_epochs = 3
train_steps = len(train_dataset)
eval_steps = len(eval_dataset)

In [43]:
for epoch in range(num_epochs):
    # Training
    print('TRAINING')
    print(f"Epoch {epoch+1}/{num_epochs}")
    total_loss = 0.0
    accuracy_metric.reset_states()
    for step, batch in enumerate(train_dataset):
        loss = train_step(batch)
        total_loss += tf.reduce_mean(loss).numpy()
        if (step + 1) % 100 == 0:
            print(f"Step {step+1}/{train_steps}, Loss: {total_loss / (step+1):.4f}, Accuracy: {accuracy_metric.result().numpy():.4f}")
    
    # Evaluation
    print('EVALUATION')
    total_loss = 0.0
    accuracy_metric.reset_states()
    for step, batch in enumerate(eval_dataset):
        loss = eval_step(batch)
        total_loss += tf.reduce_mean(loss).numpy()
    
    print(f"Validation Loss: {total_loss / eval_steps:.4f}, Validation Accuracy: {accuracy_metric.result().numpy():.4f}")

TRAINING
Epoch 1/3
Step 100/754, Loss: 2.9951, Accuracy: 0.6227
Step 200/754, Loss: 2.0151, Accuracy: 0.7241
Step 300/754, Loss: 1.6826, Accuracy: 0.7564
Step 400/754, Loss: 1.5234, Accuracy: 0.7707
Step 500/754, Loss: 1.4276, Accuracy: 0.7794
Step 600/754, Loss: 1.3637, Accuracy: 0.7860
Step 700/754, Loss: 1.3252, Accuracy: 0.7892
EVALUATION
Validation Loss: 0.9789, Validation Accuracy: 0.8265
TRAINING
Epoch 2/3
Step 100/754, Loss: 0.9598, Accuracy: 0.8302
Step 200/754, Loss: 0.9571, Accuracy: 0.8305
Step 300/754, Loss: 0.9486, Accuracy: 0.8316
Step 400/754, Loss: 0.9558, Accuracy: 0.8298
Step 500/754, Loss: 0.9559, Accuracy: 0.8300
Step 600/754, Loss: 0.9609, Accuracy: 0.8292
Step 700/754, Loss: 0.9592, Accuracy: 0.8291
EVALUATION
Validation Loss: 0.9464, Validation Accuracy: 0.8313
TRAINING
Epoch 3/3
Step 100/754, Loss: 0.9786, Accuracy: 0.8276
Step 200/754, Loss: 0.8887, Accuracy: 0.8418
Step 300/754, Loss: 0.8892, Accuracy: 0.8399
Step 400/754, Loss: 0.8881, Accuracy: 0.8404
Step 

In [54]:
spotifyDataset["test"]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'],
    num_rows: 189
})

In [67]:
spotifyDFMS3["X"][1]

"How many times have you heard someone say If I had his money I'd do things my way Hmm, but little they know Hmm, it's so hard to find One man in ten with a satisfied mind. Hmm, once I was wading in fortune and fame Everything that I dreamed of to get a start in lifes game But"

In [66]:
spotifyDFMS3["Y"][1]

"suddenly it happened Hmm, I lost every dime But I'm richer by far with a satisfied mind. Hmm, when my life is over and my time has run out My friends and my loved ones I'll leave there ain't no doubt But one things for certain When it comes my time I'll leave this old world with a satisfied mind."

In [65]:
inputs = tokenizer(spotifyDFMS3["X"][1],return_tensors="tf").input_ids
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
tokenizer.decode(outputs[0], skip_special_tokens=True)

"Hmm, I'd live the world and be on it Hmm, but little they know Hmm, it's so hard to find One man in ten with a satisfied mind"